In [4]:
# !pip install selenium
# !pip install pandas

In [3]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from datetime import datetime
from bs4 import BeautifulSoup as bs
import pandas as pd
import time

In [5]:
# !dir/w

In [7]:
# Chrome 옵션 설정
options = Options()
Options.binary_location = "./chrome-win64/chrome-win64/chrome.exe" # chrome 경로 설정
options.add_argument("--no-sandbox") # 샌드박스 설정
options.add_argument("--no-disable-dev-shm-usage") # 공유 메모리 설정
options.add_argument("--window-size=1920,1080") #화면크기설정, 반응형 웹 대비
#options.add_argument("--no-disable-gpu") #gpu 비활성화

#chrome 드라이버 설정
service = Service("chromedriver.exe")
driver = webdriver.Chrome(service=service, options=options)
driver.get("https://www.starbucks.co.kr/")
print(driver.title)

# 요소찾기
hover_element = driver.find_element(By.CSS_SELECTOR, "#gnb > div > nav > div > ul > li.gnb_nav03 > h2 > a")
menu_option = driver.find_element(By.CSS_SELECTOR, "#gnb > div > nav > div > ul > li.gnb_nav03 > div > div > div > ul:nth-child(1) > li.gnb_sub_ttl > a")

# 마우스 호버 엑션
actions = ActionChains(driver)
actions.move_to_element(hover_element).perform()

# 약간의 시간을 기다린 후 클릭
time.sleep(1)
menu_option.click()
time.sleep(5)
print("매장검색중")

# 입력창 선택 후 검색
driver.find_element(By.CSS_SELECTOR,"#container > div > form > fieldset > div > section > article.find_store_cont > article > header.loca_search > h3 > a").click() # '지역검색' 을 선택
print("지역검색중")
time.sleep(1)
driver.find_element(By.CSS_SELECTOR,"#container > div > form > fieldset > div > section > article.find_store_cont > article > article:nth-child(4) > div.loca_step1 > div.loca_step1_cont > ul > li:nth-child(6) > a").click() # '부산'을 클릭
print("부산검색중")
time.sleep(1)
driver.find_element(By.CSS_SELECTOR,"#mCSB_2_container > ul > li:nth-child(1) > a").click() # '부산'을 클릭
print("부산전체로드중")
time.sleep(1)
html = driver.page_source
# print(html)
# driver.quit()  # 브라우저 닫기

FileNotFoundError: [Errno 2] No such file or directory: 'chromedriver.exe'

In [ ]:
# 현재 페이지 정보 가져오기
star_list=[]
soup = bs(html, 'html.parser')
# 본문내용 가져오기
contents = soup.select("#mCSB_3_container > ul > li")

for c in contents:
    result = [ c['data-name'], c['data-lat'], c['data-long'], c.select_one('p.result_details').text , c.select_one('i')['class'][0] ]
    star_list.append(result)    
print(star_list)

In [ ]:
# star_list

In [ ]:
columns = ['이름','위도','경도','주소','타입']
sb = pd.DataFrame(star_list , columns=columns)
sb.head()

In [ ]:
sb.info()

In [ ]:
timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
csv_path = f'starbucks_{timestamp}.csv'
sb.to_csv(csv_path, index=True)
print("csv 파일 생성완료.")

In [ ]:
# !pip install sqlalchemy
# !pip install mysql-connector-python

In [ ]:
# mysql 에서 데이터 가져와 dataframe 로 변환하기
import mysql.connector
from sqlalchemy import create_engine

user = "root"
password = "1234"
host = "210.119.14.56"
port = "3306"
database = "jh0521"

total = f"{user}:{password}@{host}:{port}/{database}"
print(total)
engine = create_engine('mysql+mysqlconnector://'+total)

query = "SELECT * FROM STAR_STORE" # 테이블이름 주의
df = pd.read_sql(query, engine)
df.head()

In [ ]:
df = df[['storename','latitude','longitude','location','type']]
df.head()

In [ ]:
df.columns =  ['이름','위도','경도','주소','타입']
df.head()